In [1]:
import pandas as pd

In [2]:
df =pd.read_csv('SMSSpamCollection',sep='\t',names=['label','meassgae'])

In [3]:
df.head()

,label,meassgae
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
df.shape

(5572, 2)

In [5]:
df['meassgae'].iloc[50]

'What you thinked about me. First time you saw me in class.'

In [6]:
!pip install nltk

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.


In [6]:
# text preprocessing
import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/labsuser/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/labsuser/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/labsuser/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [7]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

In [8]:
stem=PorterStemmer()
lemmatizer=WordNetLemmatizer()

In [9]:
corpus=[]
for i in range(0,len(df)):
  msg=re.sub('[^a-zA-Z]',' ',df['meassgae'][i])
  msg=msg.lower()
  msg=msg.split()
  msg=[stem.stem(word) for word in msg if not word in set(stopwords.words('english'))]
  msg=' '.join(msg)
  corpus.append(msg)

In [11]:
corpus

['go jurong point crazi avail bugi n great world la e buffet cine got amor wat',
 'ok lar joke wif u oni',
 'free entri wkli comp win fa cup final tkt st may text fa receiv entri question std txt rate c appli',
 'u dun say earli hor u c alreadi say',
 'nah think goe usf live around though',
 'freemsg hey darl week word back like fun still tb ok xxx std chg send rcv',
 'even brother like speak treat like aid patent',
 'per request mell mell oru minnaminungint nurungu vettam set callertun caller press copi friend callertun',
 'winner valu network custom select receivea prize reward claim call claim code kl valid hour',
 'mobil month u r entitl updat latest colour mobil camera free call mobil updat co free',
 'gonna home soon want talk stuff anymor tonight k cri enough today',
 'six chanc win cash pound txt csh send cost p day day tsandc appli repli hl info',
 'urgent week free membership prize jackpot txt word claim c www dbuk net lccltd pobox ldnw rw',
 'search right word thank breather

In [12]:
corpus[0]

'go jurong point crazi avail bugi n great world la e buffet cine got amor wat'

In [13]:
#BOG-count vectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [14]:
cv=CountVectorizer(ngram_range=(3,3),max_features=2500,binary=True)
x=cv.fit_transform(corpus)

In [15]:
x.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [16]:
x.shape

(5572, 2500)

In [12]:
y=pd.get_dummies(df['label'])

In [18]:
y.head()

,ham,spam
0,1,0
1,1,0
2,0,1
3,1,0
4,1,0


In [13]:
y=y.iloc[:,1]

In [20]:
y.head()

0    0
1    0
2    1
3    0
4    0
Name: spam, dtype: uint8

In [21]:
y.shape

(5572,)

In [22]:
#train test split

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(x,y,test_size=0.20,random_state=42)

In [23]:
X_train,y_train

(<4457x2500 sparse matrix of type '<class 'numpy.int64'>'
 	with 6567 stored elements in Compressed Sparse Row format>,
 1978    1
 3989    0
 3935    0
 4078    0
 4086    1
        ..
 3772    0
 5191    0
 5226    0
 5390    0
 860     0
 Name: spam, Length: 4457, dtype: uint8)

In [24]:
X_train.shape

(4457, 2500)

In [25]:
X_test.shape

(1115, 2500)

In [26]:
from sklearn.naive_bayes import MultinomialNB

In [27]:
model=MultinomialNB()
model.fit(X_train,y_train)

MultinomialNB()

In [28]:
y_pred=model.predict(X_test)

In [29]:
y_pred

array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)

In [10]:
from sklearn.metrics import classification_report,accuracy_score

In [31]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.95      1.00      0.97       966
           1       1.00      0.62      0.77       149

    accuracy                           0.95      1115
   macro avg       0.97      0.81      0.87      1115
weighted avg       0.95      0.95      0.94      1115



In [32]:
accuracy_score(y_test,y_pred)

0.9497757847533632

In [33]:
X_train.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [42]:
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier()
rf.fit(X_train,y_train)
y_pred=rf.predict(X_test)
accuracy_score(y_test,y_pred)

0.9479820627802691

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
cv=TfidfVectorizer(max_features=2500)
X=cv.fit_transform(corpus).toarray()

In [53]:
X.shape

(5572, 2500)

In [15]:

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20,random_state=42)

In [16]:
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier()
rf.fit(X_train,y_train)
y_pred=rf.predict(X_test)
accuracy_score(y_test,y_pred)

0.97847533632287

In [18]:
from sklearn.naive_bayes import MultinomialNB
model=MultinomialNB()
model.fit(X_train,y_train)
y_pred=model.predict(X_test)
accuracy_score(y_test,y_pred)

0.9811659192825112

In [19]:
## word2vec

!pip install gensim

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.


In [20]:
import gensim

In [30]:
from gensim.models import Word2Vec,keyedvectors
from gensim.utils import simple_preprocess

In [23]:
from nltk import sent_tokenize

In [25]:
word=[]
for i in corpus:
    token=sent_tokenize(i)
    for i in token:
        word.append(simple_preprocess(i))
    

In [27]:
import gensim

In [36]:
model=gensim.models.Word2Vec(word,window=3,min_count=2)
model=gensim.models.Word2Vec(word,window=5,min_count=2)

In [38]:
model.corpus_count

5564

In [43]:
model.wv.similar_by_word('vijay')

[('island', 0.9903610944747925),
 ('qualiti', 0.9903380870819092),
 ('earth', 0.9902939796447754),
 ('wiv', 0.9900822639465332),
 ('india', 0.9900745153427124),
 ('privat', 0.9900628328323364),
 ('digit', 0.9900386333465576),
 ('dude', 0.9900072813034058),
 ('silent', 0.9900053143501282),
 ('alright', 0.989953339099884)]

In [41]:
model.wv['hai'].shape

(100,)

In [44]:
model.epochs

5

In [54]:
model.wv.vocab

{'go': <gensim.models.keyedvectors.Vocab at 0x7f75851df750>,
 'point': <gensim.models.keyedvectors.Vocab at 0x7f75851df310>,
 'crazi': <gensim.models.keyedvectors.Vocab at 0x7f75851df2d0>,
 'avail': <gensim.models.keyedvectors.Vocab at 0x7f75851df0d0>,
 'bugi': <gensim.models.keyedvectors.Vocab at 0x7f75851df510>,
 'great': <gensim.models.keyedvectors.Vocab at 0x7f75850bb410>,
 'world': <gensim.models.keyedvectors.Vocab at 0x7f75850bb150>,
 'la': <gensim.models.keyedvectors.Vocab at 0x7f75850bbd90>,
 'buffet': <gensim.models.keyedvectors.Vocab at 0x7f75851df490>,
 'cine': <gensim.models.keyedvectors.Vocab at 0x7f75850bb3d0>,
 'got': <gensim.models.keyedvectors.Vocab at 0x7f75850bb250>,
 'wat': <gensim.models.keyedvectors.Vocab at 0x7f75850bb750>,
 'ok': <gensim.models.keyedvectors.Vocab at 0x7f75850bbf10>,
 'lar': <gensim.models.keyedvectors.Vocab at 0x7f75850bbc50>,
 'joke': <gensim.models.keyedvectors.Vocab at 0x7f75850bb290>,
 'wif': <gensim.models.keyedvectors.Vocab at 0x7f75850bb3